# NAME: ANSER GHAFFAR
# ROLL NO: SU92-BSDSM-F23-032
# SECTION: BSDS 4A
# PAI LAB TASK 6

In [ ]:
import cv2
import numpy as np
import torch
import folium
from datetime import datetime
from ultralytics import YOLO
from flask import Flask, render_template, Response

In [ ]:
app = Flask(__name__)
model = YOLO("yolov8n.pt")  
cap = cv2.VideoCapture(0)

alert_locations = []
base_lat, base_lon = 31.5204, 74.3587  

def generate_frames():
    frame_count = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break 
        
        results = model(frame)
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()
                label = result.names[int(box.cls[0].item())]
                
                if conf > 0.5:  
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    lat = base_lat + np.random.uniform(-0.01, 0.01)
                    lon = base_lon + np.random.uniform(-0.01, 0.01)
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    alert_locations.append((lat, lon, label, timestamp))
        
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        
        frame_count += 1
        if frame_count >= 100: 
            break


In [ ]:
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/stop')
def stop():
    cap.release()
    
    m = folium.Map(location=(base_lat, base_lon), zoom_start=12)

    for lat, lon, label, timestamp in alert_locations:
        popup_text = f"{label} detected at {timestamp}"
        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            icon=folium.Icon(color="red", icon="paw")
        ).add_to(m)

    map_path = "static/herd_alert_map.html"
    m.save(map_path)
    return f"Detection stopped. <br> <a href='{map_path}' target='_blank'>View Animal Detection Map</a>"

if __name__ == '__main__':
    app.run(debug=True)
